In [1]:
import csv

In [2]:
!pip3 install neo4j


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
AUTH = ('neo4j', 'neo4j1234')

In [4]:
def insertAirport(driver, airport_id, name, city, country, IATA, ICAO):
    query = 'CREATE (:Airport {airport_id: $airport_id, name: $name, city: $city, country: $country, IATA: $IATA, ICAO: $ICAO})'
    driver.execute_query(query, airport_id=airport_id, name=name, city=city, country=country, IATA=IATA, ICAO=ICAO)
    

In [5]:
def insertAirline(driver, airline_id, name, IATA, ICAO, country, active):
    query = 'CREATE (:Airline {airline_id: $airline_id, name: $name, IATA: $IATA, ICAO: $ICAO, country: $country, active: $active})'
    driver.execute_query(query, airline_id=airline_id, IATA=IATA, ICAO=ICAO, name=name, country=country, active=active=='Y')

In [6]:
def insertRoute(driver, airline_name, airline_id, source_airport_id, dest_airport_id, codeshare):

    query = 'match (s:Airport), (d:Airport) where s.airport_id = $source_airport_id and d.airport_id = $dest_airport_id create (s)-[r:Route{airline_name: $airline_name, airline_id: $airline_id, codeshare: $codeshare}]->(d)'
    
    driver.execute_query(query, source_airport_id = source_airport_id, dest_airport_id=dest_airport_id, airline_name = airline_name, airline_id=airline_id, codeshare=codeshare=='Y')

In [8]:
# Use to filter routes
selected_airport_ids = []
selected_airline_ids = []

In [9]:
reduced_airport = "/Users/helenhoang/Downloads/TDT-bigdata-1/reduced_airports.csv"

In [17]:
def readAirport():
    with open('airports.csv') as f, open(reduced_airport, mode='w', newline='') as o:
        reader = csv.reader(f)
        writer = csv.writer(o)

        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                [airport_id, name, city, country, IATA, ICAO, _, _, _, _, _, _, dtype, _] = row
                
                #Reduction: Only select airports in Canada or United States
                if dtype == 'airport' and country in ['Canada', 'United States']:
                    if '\\N' not in [airport_id, name, city, country, IATA, ICAO]:
                        selected_airport_ids.append(airport_id)
                        print(row)
                        # Write the fields to the CSV file
                        reduced_row = [row[0], row[1], row[2], row[3], row[4], row[5]]
                        writer.writerow(reduced_row)
                        
                        # add data in Neo4j
                        #insertAirport(driver, airport_id, name, city, country, IATA, ICAO)

                # print(id, name, city, country, IATA, ICAO, dtype)
            
        driver.close()
    f.close()

In [18]:
readAirport()

['21', 'Sault Ste Marie Airport', 'Sault Sainte Marie', 'Canada', 'YAM', 'CYAM', '46.48500061035156', '-84.5093994140625', '630', '-5', 'A', 'America/Toronto', 'airport', 'OurAirports']
['24', 'St. Anthony Airport', 'St. Anthony', 'Canada', 'YAY', 'CYAY', '51.3918991089', '-56.083099365200006', '108', '-3.5', 'A', 'America/St_Johns', 'airport', 'OurAirports']
['25', 'Tofino / Long Beach Airport', 'Tofino', 'Canada', 'YAZ', 'CYAZ', '49.079833', '-125.775583', '80', '-8', 'A', 'America/Vancouver', 'airport', 'OurAirports']
['26', 'Kugaaruk Airport', 'Pelly Bay', 'Canada', 'YBB', 'CYBB', '68.534401', '-89.808098', '56', '-7', 'A', 'America/Edmonton', 'airport', 'OurAirports']
['27', 'Baie Comeau Airport', 'Baie Comeau', 'Canada', 'YBC', 'CYBC', '49.13249969482422', '-68.20439910888672', '71', '-5', 'A', 'America/Toronto', 'airport', 'OurAirports']
['28', 'CFB Bagotville', 'Bagotville', 'Canada', 'YBG', 'CYBG', '48.33060073852539', '-70.99639892578125', '522', '-5', 'A', 'America/Toronto',

In [20]:
len(selected_airport_ids)

1631

In [24]:
def readReduced_Airport():
    with open('reduced_airports.csv') as f:
        reader = csv.reader(f)

        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                [airport_id, name, city, country, IATA, ICAO] = row
                #print(row)
                insertAirport(driver, airport_id, name, city, country, IATA, ICAO)

        driver.close()
    f.close()

In [25]:
readReduced_Airport()

In [26]:
reduced_airline = "/Users/helenhoang/Downloads/TDT-bigdata-1/reduced_airlines.csv"

In [27]:
def readAirline():
    with open('airlines.csv') as f, open(reduced_airline, mode='w', newline='') as o:
        reader = csv.reader(f)
        writer = csv.writer(o)
        
        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                #Airline(airline ID, name, Alias, IATA, ICAO, Callsign, Country, Active)
                [airline_id, name, _, IATA, ICAO, _, country, active] = row
                
                #Reduction: Only select airlines in Canada or United States
                if country in ['Canada', 'United States']:
                    if '\\N' not in [airline_id, name, IATA, ICAO, country, active]:
                        selected_airline_ids.append(airline_id)
                        print(row)
                        # Write the fields to the CSV file
                        reduced_row = [row[0], row[1], row[3], row[4], row[6], row[7]]
                        writer.writerow(reduced_row)

                        # add data in Neo4j
                        #insertAirline(driver, airline_id, name, IATA, ICAO, country, active)

        driver.close()
    f.close()

In [28]:
readAirline()

['2', '135 Airways', '\\N', '', 'GNL', 'GENERAL', 'United States', 'N']
['9', '3D Aviation', '\\N', '', 'SEC', 'SECUREX', 'United States', 'N']
['10', '40-Mile Air', '\\N', 'Q5', 'MLA', 'MILE-AIR', 'United States', 'Y']
['12', '611897 Alberta Limited', '\\N', '', 'THD', 'DONUT', 'Canada', 'N']
['17', 'Aero Aviation Centre Ltd.', '\\N', '', 'AAD', 'SUNRISE', 'Canada', 'N']
['22', 'Aloha Airlines', '\\N', 'AQ', 'AAH', 'ALOHA', 'United States', 'Y']
['23', 'Alaska Island Air', '\\N', '', 'AAK', 'ALASKA ISLAND', 'United States', 'N']
['24', 'American Airlines', '\\N', 'AA', 'AAL', 'AMERICAN', 'United States', 'Y']
['25', 'Aviation Management Corporation', '\\N', '', 'AAM', 'AM CORP', 'United States', 'N']
['26', 'Atlantis Airlines (USA)', '\\N', '', 'AAO', 'ATLANTIS AIR', 'United States', 'N']
['31', 'Astro Air International', '\\N', '', 'AAV', 'ASTRO-PHIL', 'United States', 'N']
['35', 'Allegiant Air', '\\N', 'G4', 'AAY', 'ALLEGIANT', 'United States', 'Y']
['36', 'Angus Aviation', '\\N', 

In [29]:
len(selected_airline_ids)

1390

In [30]:
def readReduced_Airline():
    with open('reduced_airlines.csv') as f:
        reader = csv.reader(f)

        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                [airline_id, name, IATA, ICAO, country, active] = row
                #print(row)
                insertAirline(driver, airline_id, name, IATA, ICAO, country, active)

        driver.close()
    f.close()

In [31]:
readReduced_Airline()

In [32]:
reduced_routes = "/Users/helenhoang/Downloads/TDT-bigdata-1/reduced_routes.csv"

In [36]:
def readRoute():
    count2 = 0
    with open('routes.csv') as f, open(reduced_routes, mode='w', newline='') as o:
        reader = csv.reader(f)
        writer = csv.writer(o)
        
        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                #Route(Airline, Airline ID, Source airport, Source airport ID, Destination airport, Destination airport ID, 
                # Codeshare, Stops, Equipment)
                [airline_name, airline_id, _, source_airport_id, _, dest_airport_id, codeshare, stops, _] = row
                
                if '\\N' not in [airline_name, airline_id, source_airport_id, dest_airport_id]:
                    # Since most data has stops = 0, we skip rows with stops > 0
                    if stops == "0" and airline_id in selected_airline_ids and source_airport_id in selected_airport_ids and dest_airport_id in selected_airport_ids:
                        print(row)
                        count2 += 1
                        reduced_row = [row[0], row[1], row[3], row[5], row[6]]
                        writer.writerow(reduced_row)

                        # add data in Neo4j
                        #insertRoute(driver, airline_name, airline_id, source_airport_id, dest_airport_id, codeshare)
        driver.close()
    print(count2)
    f.close()

In [37]:
readRoute()

['4B', '20565', 'LAS', '3877', 'LAX', '3484', '', '0', 'PL2']
['4B', '20565', 'LAX', '3484', 'LAS', '3877', '', '0', 'PL2']
['4N', '341', 'YEG', '49', 'YXY', '176', '', '0', '737']
['4N', '341', 'YLW', '87', 'YVR', '156', '', '0', '737']
['4N', '341', 'YLW', '87', 'YXY', '176', '', '0', '737']
['4N', '341', 'YOW', '100', 'YZF', '196', '', '0', '737']
['4N', '341', 'YVR', '156', 'YLW', '87', '', '0', '737']
['4N', '341', 'YVR', '156', 'YXY', '176', '', '0', '737']
['4N', '341', 'YXY', '176', 'YLW', '87', '', '0', '737']
['4N', '341', 'YXY', '176', 'YVR', '156', '', '0', '737']
['4N', '341', 'YXY', '176', 'YYC', '178', '', '0', '737']
['4N', '341', 'YXY', '176', 'YZF', '196', '', '0', '737']
['4N', '341', 'YYC', '178', 'YEG', '49', '', '0', '737']
['4N', '341', 'YZF', '196', 'YOW', '100', '', '0', '737']
['4N', '341', 'YZF', '196', 'YXY', '176', '', '0', '737']
['5T', '1623', 'YBK', '29', 'YCS', '5487', 'Y', '0', 'ATR']
['5T', '1623', 'YBK', '29', 'YXN', '5534', 'Y', '0', 'ATR']
['5T', '

In [38]:
def readReduced_Route():
    with open('reduced_routes.csv') as f:
        reader = csv.reader(f)

        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            for row in reader:
                [airline_name, airline_id, source_airport_id, dest_airport_id, codeshare] = row
                #print(row)
                insertRoute(driver, airline_name, airline_id, source_airport_id, dest_airport_id, codeshare)

        driver.close()
    f.close()

In [39]:
readReduced_Route()

In [40]:
import psutil

In [41]:
# Function to check CPU and memory usage
def check_resource_usage():
    process = psutil.Process()
    cpu_percent = process.cpu_percent()
    memory_info = process.memory_info()
    
    print(f"CPU Usage: {cpu_percent}%")
    print(f"Memory Usage: {memory_info.rss / (1024 * 1024):.2f} MB")

# Call your functions here
check_resource_usage()

CPU Usage: 0.0%
Memory Usage: 31.80 MB


In [42]:
# Check RAM usage
ram_usage = psutil.virtual_memory()
print(f"Total RAM: {ram_usage.total / (1024 ** 3):.2f} GB")
print(f"Available RAM: {ram_usage.available / (1024 ** 3):.2f} GB")
print(f"Used RAM: {ram_usage.used / (1024 ** 3):.2f} GB")
print(f"RAM Usage (%): {ram_usage.percent}%")

# Check CPU usage
cpu_usage = psutil.cpu_percent(interval=1, percpu=True)
print(f"CPU Usage (%): {cpu_usage}")

Total RAM: 16.00 GB
Available RAM: 2.75 GB
Used RAM: 6.57 GB
RAM Usage (%): 82.8%
CPU Usage (%): [46.5, 47.0, 36.4, 19.8, 15.0, 15.2, 10.0, 6.9]
